In [1]:
from IPython.display import display

import pandas as pd
import numpy as np
import glob
import re

## We read in several census files in order to get geographic, demographic, economic, etc. data by county

#### First we create a table that can parse the census data columns (via Mastdata.csv)

In [2]:
# The Mastdata.csv contains column code to English description of the data
mastdata = pd.read_csv('raw_census_data/Mastdata.csv')

In [3]:
mastdata.head()

,Item_Id,Item_Description,Unit_Indicator,Decimal_Indicator,US_Total,Source1,Source11
0,AFN110197D,Accommodation and Food Services: total (NAICS ...,ABS,0,545068.0,CENSUS-ECON,NaN
1,AFN110202D,Accommodation and Food Services: total (NAICS ...,ABS,0,565590.0,CENSUS-ECON,NaN
2,AFN110207D,Accommodation and Food Services: total (NAICS ...,ABS,0,634361.0,CENSUS-ECON,NaN
3,AFN120197D,Accommodation and Food Services: total (NAICS ...,TH$,0,350399194.0,CENSUS-ECON,NaN
4,AFN120202D,Accommodation and Food Services: total (NAICS ...,TH$,0,449498718.0,CENSUS-ECON,NaN


In [4]:
# Create an object that can convert the column code to a year and description
mastkeys = dict(zip(list(mastdata["Item_Id"]), list(mastdata["Item_Description"])))
for key, val in mastkeys.items():
    year = int("{}{}{}".format(key[6], "9" if key[6] == "1" else "0", key[7:9]))
    # val = val.replace(str(year), '')
    mastkeys[key] = (year, val)

#### Then define a set of functions and associated arguments for handling different csv datasets

In [5]:
# Define functions to load the csvs into a single dataframe, and to process the census data into a clean format
def load_census_csvs(code):
    df = pd.DataFrame()
    for fn in glob.glob('raw_census_data/{}0?.csv*'.format(code)):
        if len(df) == 0:
            df = pd.read_csv(fn)
            try:
                df.drop("Areaname", axis=1, inplace=True)
            except:
                try:
                    df.drop("Area_name", axis=1, inplace=True)
                except:
                    pass
        else:
            curr_data = pd.read_csv(fn)
            try:
                curr_data.drop("Areaname", axis=1, inplace=True)
            except:
                try:
                    curr_data.drop("Area_name", axis=1, inplace=True)
                except:
                    pass
            df = df.join(curr_data.set_index("STCOU"), on=["STCOU"], how="left")
    display(df.head())
    return df

def process_census_df(df, mastkeys, only_most_recent=True, save=False, code='', kwsets=[], nkws=[],
                      veto_func=lambda x:False, name_func=lambda x:x, derived_cols_func=lambda x:x):
    # Retain only interesting columns. only_most_recent takes newest of a measurement, and kws looks for keywords in the full description
    mr_key = {}
    for column in df.columns:
        # Ignore if not a data column
        if column == 'STCOU':
            pass
        elif column not in list(mastkeys.keys()):
            df.drop(column, axis=1, inplace=True)
        # remove any with a negative keyword
        elif any([nkw in mastkeys[column][1] for nkw in nkws]):
            df.drop(column, axis=1, inplace=True)
        # remove any without a complete set of positive kws
        elif not any([all([kw in mastkeys[column][1] for kw in kwset]) for kwset in kwsets]):
            df.drop(column, axis=1, inplace=True)
        # arbitrary veto function
        elif veto_func(mastkeys[column][1]):
            df.drop(column, axis=1, inplace=True)
        # recency filter
        elif column[:6] in mr_key and only_most_recent: 
            if mr_key[column[:6]][6:9] > column[6:9]:
                df.drop(column, axis=1, inplace=True)
            else:
                df.drop(mr_key[column[:6]], axis=1, inplace=True)
                mr_key[column[:6]] = column
        else:
            mr_key[column[:6]] = column

    # Rename the columns based on the full description
    overlap_names = {key: name_func(name) for key, (year, name) in mastkeys.items() if key in df.columns}
    overlap_names.update({"STCOU": "FIPS"})
    df = df.rename(columns=overlap_names)
    data_cols = list(df.columns)
    data_cols.remove('FIPS')
    df = df[['FIPS'] + sorted(data_cols)]
    
    # Apply some feature processing if needed
    derived_cols_func(df)
    
    if save:
        df.to_csv('proc_census_data/{}.csv'.format(code), sep=',')
    display(df.head())
    return df

In [10]:
# Define special processing functions for the age data
age_re = re.compile("(\d{1,2}) to (\d{1,2})")
race_re = re.compile(":\s([^,]*),")
crime_re = re.compile("\s(\w+\scrimes)\s")
rem_crap_re = re.compile("\s(\([^\)]*\))+|\s\d{4}(\s?-\s?\d{4})*")

def normalize_column(df, name, new_name=None):
    if not new_name:
        new_name = "Normalized {}".format(name)
    df[new_name] = df[name] / np.mean(df[name].values)
    df.drop([name], axis=1, inplace=True)
    
def column_ratio(df, numerators, denominators, name, to_remove=None):
    num = sum([df[col] for col in numerators])
    den = sum([df[col] for col in denominators])
    df[name] = (num / den).fillna(0)
    if to_remove is None:
        to_remove = list(set(numerators + denominators))
    df.drop(to_remove, axis=1, inplace=True)
    
def rem_crap_naming(name):
    try:
        return rem_crap_re.sub('', name)
    except:
        return 'fail'

def age_range_veto(name):
    try:
        lower, higher = map(int, age_re.search(name).groups())
        return higher - lower > 5
    except:
        if 'under' in name and ('5' not in name and '18' not in name):
            return True
        elif 'over' in name and ('18' not in name and '85' not in name):
            return True
        return False

def age_range_naming(name):
    try:
        lower, higher = map(int, age_re.search(name).groups())
        return "Age {:0>2d} to {:0>2d}".format(lower, higher)
    except:
        if 'under' in name:
            return "Age 00 to {:0>2d}".format(int(re.search(r"under (\d{1,2})", name).groups()[0]))
        if 'over' in name:
            return "Age {:0>2d} and over".format(int(re.search(r"(\d{1,2}) years and over", name).groups()[0]))
        return name
    
def age_derived_columns(df):
    column_ratio(df, ['Age 18 and over'], ['Age 00 to 18', 'Age 18 and over'], 'Voting Age Fraction', to_remove=['Age 00 to 18'])
    
def agr_derived_columns(df):
    column_ratio(df, ['Value of farm products sold - crops'], ['Value of farm products sold - total'],
                 'Crop fraction of farmland', to_remove=['Value of farm products sold - crops'])
    column_ratio(df, ['Value of farm products sold - total'], ['Cropland - total'], 
                 'Cropland Value per Acre', to_remove=['Value of farm products sold - total'])
    normalize_column(df, 'Cropland - total')
    
def pov_derived_columns(df):
    normalize_column(df, 'Median household income 2009', 'Normalized Median Income')
    df['Poverty Fraction'] = df['People of all ages in poverty - percent 2009']
    df.drop(['People of all ages in poverty - percent 2009'], axis=1, inplace=True)
    
def land_derived_columns(df):
    normalize_column(df, 'Land area')
    
def pop_derived_columns(df):
    column_ratio(df, ['Urban population'], ['Resident population'], 'Urban Fraction', to_remove=['Urban population'])
    
def race_naming(name):
    try:
        return race_re.search(name).groups()[0].replace(' alone','')
    except:
        return name

def crime_naming(name):
    try:
        return crime_re.search(name).groups()[0]
    except:
        return name

In [11]:
# processing arguments for different datasets
land_args = {'code': "LND",
             'kwsets': [['Land']],
             'name_func': lambda x:x[:9],
             'derived_cols_func': land_derived_columns}

age_args = {'code': "AGE",
            'kwsets': [['years', 'complete', '2010', '18']],
            'nkws': ['percent', 'Median'],
            'veto_func': age_range_veto,
            'name_func': age_range_naming,
            'derived_cols_func': age_derived_columns}

agr_args = {'code': "AGN",
            'kwsets': [['Cropland', 'total'], ['products', 'total'], ['products', 'crops']],
            'nkws': ['1997'],
            #'veto_func': agr_veto,
            'name_func': rem_crap_naming,
            'derived_cols_func': agr_derived_columns
           }

poverty_args = {'code': "IPE",                
                'nkws': ['number', 'under', 'families'],
                'name_func': lambda x:x,
                'derived_cols_func': pov_derived_columns}

race_args = {'code': "RHI",
             'kws': [['April', 'percent']],
             'nkws': ['estimates'],
             'name_func': race_naming}

crime_args = {'code': "CRM",
              'veto_func': lambda x: not ('property' in x or 'violent' in x),
              'name_func': crime_naming,              
             }

pop_args = {'code': "POP",
            'kwsets': [['Urban population'], ['square mile'], ['Resident population', '2010'], ['Percent change']],
            #'nkws': ['citizenship']
            #'veto_func': lambda x: not ('property' in x or 'violent' in x),
            'name_func': rem_crap_naming,
            'derived_cols_func': pop_derived_columns
           }

#### Now run the full processing flow by choosing the arguments needed for your dataset

In [12]:
#Choose one of the above argument sets
args = age_args
args.update({'save': True})
# args.update({'only_most_recent': False})

full_data = load_census_csvs(args['code'])
full_data = process_census_df(full_data, mastkeys, **args)

,STCOU,AGE890200F,AGE890200D,AGE890200N1,AGE890200N2,AGE900200F,AGE900200D,AGE900200N1,AGE900200N2,AGE900201F,...,AGE430203N1,AGE430203N2,AGE430204F,AGE430204D,AGE430204N1,AGE430204N2,AGE430205F,AGE430205D,AGE430205N1,AGE430205N2
0,0,0,4239674,0,0,0,4270521,0,0,0,...,0,0,0,20259538,0,0,0,19846368,0,0
1,1000,0,67301,0,0,0,67697,0,0,0,...,0,0,0,300648,0,0,0,295756,0,0
2,1001,0,428,0,0,0,434,0,0,0,...,0,0,0,3186,0,0,0,3153,0,0
3,1003,0,2164,0,0,0,2193,0,0,0,...,0,0,0,9602,0,0,0,9578,0,0
4,1005,0,512,0,0,0,513,0,0,0,...,0,0,0,2130,0,0,0,2131,0,0


,FIPS,Age 18 and over,Voting Age Fraction
0,0,234564071,0.759733
1,1000,3647277,0.763071
2,1001,39958,0.732220
3,1003,140367,0.770126
4,1005,21442,0.780930


In [13]:
full_data[full_data['Crop fraction of farmland'].isnull()]

,FIPS,Crop fraction of farmland,Cropland Value per Acre,Normalized Cropland - total
